Create data for figure 3 and save to respective csv files

In [28]:
# Load the BK functions, the x_grid(Ca, V) that we evaluate over and the initial condition phi0
include("../BK_functions/bk_setup_script.jl"); 

In [29]:
# Run the BK function reduction (reduced models are handcoded within BK_functions)
#=
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10, 11, 12, 13, 14, 15], 
move_dir=[-1, -1, 1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce)
=#

# This takes about ~3 hours on my laptop (last iter especially slow), alternatively load figure3.jld which stores this result
using JLD 
outp = load("figure3.jld", "outp");

# Figure 3a - Model output at reductions

* Panel 1 - Model output for 12 uM of Calcium for different iterations
* Panel 2 - Total cost for different iterations

In [33]:
# Panel 1 - output

ca = 12.0*1e-6
ca_str = @sprintf("%1.1f", ca*1e6)

# Generate grid to evaluate over
x_grid_cur = Array(Any, (1*length(V),))
iterind = 0;
for j1 in product([ca], V)
    iterind += 1;
    x_grid_cur[iterind] = collect(j1)
end


# Get the function values for original model
tmp = BK_simulator(exp(outp[1][1,1]), hcat(x_grid_cur...), model_id=10)
writecsv("CSV/figure_3a_panel1_iter0.csv", hcat(V[:], tmp[:]))

for model_red = 1:size(outp[1],1)
    # Get the function values for reduced models
    tmp = BK_simulator(exp(outp[1][model_red,2]), hcat(x_grid_cur...), model_id=(9+model_red))
    writecsv("CSV/figure_3a_panel1_iter$(model_red).csv", hcat(V[:], tmp[:]))
end

In [35]:
# Panel 2 - Cost
writecsv("CSV/figure_3a_panel2.csv", hcat(0:size(outp[1],1), vcat(outp[2][1,1], outp[2][:,2])))

# Figure 3b - Model reduction

* Panel n - nth reduction containing each parameter curve


In [53]:
param_names = Array(Array{AbstractString},size(outp[1],1));
param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[3] = ["J0", "zJ", "Kd", "LC","LD", "EL"];
param_names[4] = ["J0", "zJ", "Kd", "CE","LD"];
param_names[5] = ["LDJ", "zJ", "Kd", "CE"];
if size(outp[1],1)>=6
    param_names[6] = ["LDJ", "zJ", "CEKd"];
end

In [52]:
for model_red = 1:size(outp[1],1)
    # Get the log param values during the reduction
    phi_red_time = outp[5][model_red]
    phi_red_vals = hcat(outp[6][model_red]...)
    phi_red_vals = phi_red_vals[1:div(size(phi_red_vals,1),2),:]
    
    # Find the converged time (the one after no parameters change)
    cutoff = 0
    for i1 = 2:size(phi_red_vals,2)
        if all(phi_red_vals[:,i1].==phi_red_vals[:,i1-1])
            cutoff = i1
            break
        end
    end
    
    # Produce the geodesic time - log(value) for each parameter for each reduction step
    for param_num in 1:size(phi_red_vals,1)
        writecsv("CSV/figure_3b_panel$(model_red)_param$(param_names[model_red][param_num]).csv", 
            hcat(phi_red_time[1:cutoff], phi_red_vals[param_num,1:cutoff][:]))
    end
end